In [2]:
#Jack Baxter
#First Agent

In [ ]:
#import helper functions from utils.py
from utils import load_env, get_api_keys
#import libs for working with images
from PIL import Image
import matplotlib.pyplot as plt

#load the .env file from utils function
load_env()

#get path to scanned image
#put image in data folder 
#initialize path
image_path = "data/front_scan.jpg"

#load image and display it
img = Image.open(image_path)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')
plt.title('3x3 baseball card scan', fontsize=8)
plt.show()



environment loaded
